In [1]:
!nvidia-smi

Thu Apr 13 10:39:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# FINETUNING PEGASUS

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [2]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")      #The punkt package provides pre-trained models for tokenizing text in several languages, including English, Spanish, French, German, and Dutch.

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
     


## LOADING THE DATASET

In [6]:
df = pd.read_csv("/kaggle/input/NLPSUM/Training.csv")
df

,Unnamed: 0,Unnamed: 0.1,content,c_summary
0,0,0,exciting news from openai they have just relea...,chatgptOpenAI has released their ChatGPT and W...
1,1,1,the first words i typed this morning git fetc...,This set of tweets discusses the recent launch...
2,2,2,i morning briefing how are universities dealin...,This set of tweets is discussing the use of Op...
3,3,3,the speed at with microsoft is executing on o...,"This set of tweets is discussing OpenAI, a com..."
4,4,4,openai integration tools for odoo editor are n...,This set of tweets discusses the integration o...
...,...,...,...,...
1955,1955,1955,Delta lying to the cstmrs like usual. Force me...,Customer is complaining about bag check and as...
1956,1956,1956,"Hi SpotifyCares, any chance this may become a ...",The customer is complaining that he cannot cro...
1957,1957,1957,Come on O2 fix your connection with Carphone W...,Customer is having an issue with the connectio...
1958,1958,1958,Raise your hand if you think VerizonSupport is...,Customer having am issue with internet as his...


In [7]:
type(df['content'])

pandas.core.series.Series

In [8]:
df['content'][0]

"exciting news from openai they have just released their chatgpt and whisper apis, boosting the capabilities and accessibility of aipowered technologies. time to test and learn to use these. chatgpt openaiai learning to access the physical world. https t.cobvsp5w2tae ai chatgpt openai microsoftlet me know your idea for chatgpt api.keep in mind that no investor is interested in a product that is dependent on another company's service. openaiopenai launches an api for chatgpt, plus dedicated capacity for enterprise customers  techcrunch  openai chatgptthe top 10 worst ai photoshop failsnexttopwriter art aiart machinelearning mlsogood artificialintelligence datascience data code mlart algorithm ai chatgpt openai digitalart generativeart midjourney dalle2medium generativeai I immediately incorporated chatgpt's api into my childI borrowed it here.I feel like it is a more natural conversation than gpt3. And earlyI still have good memories.aivtuber chatgpt openai  musk recruits team to develo

In [9]:
df['c_summary'][0]

"chatgptOpenAI has released their ChatGPT and Whisper APIs, making AIpowered technologies more accessible. The APIs have been made available to the general public, and Microsoft has enabled ChatGPT to access the physical world. Elon Musk is recruiting a team to develop a rival to OpenAI's ChatGPT. The APIs are being used in various applications, such as analytics, machine learning, IoT, flutter, and TensorFlow. People are also using the APIs to explore philosophical and moral questions."

## Sample Summary Generation Using Pretrained Model

In [11]:
pipe = pipeline('summarization', model = model_ckpt )

pipe_out = pipe(df['content'][0] )

print(pipe_out)

[{'summary_text': 'openai releases chatgpt and whisper apis .<n> chatgpt openaiai learning to access the physical world .<n> whisper apis made available to general public starting today .'}]


In [12]:
print(pipe_out[0]['summary_text'].replace("<n>", ""))

openai releases chatgpt and whisper apis . chatgpt openaiai learning to access the physical world . whisper apis made available to general public starting today .


# TESTING BEFORE FINETUNING

In [4]:
t_df = pd.read_csv("/kaggle/input/tttttt/test.csv")

In [6]:
t_df

,Unnamed: 0,source_text,target_text
0,164,summarize: collinrugg sars cov 2 was planned....,This set of tweets discusses the origin of SAR...
1,492,summarize: Where are you Smriti Irani...??Nare...,This set of tweets is discussing the recent in...
2,323,summarize: mixed day for r ashwinbecame no.1 b...,This set of tweets is discussing the performan...
3,898,summarize: it is obvious messi winning the nex...,This set of tweets is discussing Lionel Messi'...
4,732,summarize: the armed forces understand that th...,"This set of tweets discusses the stock market,..."
...,...,...,...
121,209,summarize: ISRO has successfully completed fli...,The Indian Space Research Organisation ISRO ha...
122,576,summarize: bodhdiwasofsantgaribdasji Kabir Sag...,These tweets are about the celebration of Sant...
123,186,summarize: allisonpearson you missed the point...,This set of tweets discusses the research into...
124,839,summarize: action alliance heading to court to...,The Action Alliance is heading to court to con...


In [5]:
t_df = t_df.rename(columns={"target_text":"c_summary", "source_text":"content"})

In [8]:
t_df

,Unnamed: 0,content,c_summary
0,164,summarize: collinrugg sars cov 2 was planned....,This set of tweets discusses the origin of SAR...
1,492,summarize: Where are you Smriti Irani...??Nare...,This set of tweets is discussing the recent in...
2,323,summarize: mixed day for r ashwinbecame no.1 b...,This set of tweets is discussing the performan...
3,898,summarize: it is obvious messi winning the nex...,This set of tweets is discussing Lionel Messi'...
4,732,summarize: the armed forces understand that th...,"This set of tweets discusses the stock market,..."
...,...,...,...
121,209,summarize: ISRO has successfully completed fli...,The Indian Space Research Organisation ISRO ha...
122,576,summarize: bodhdiwasofsantgaribdasji Kabir Sag...,These tweets are about the celebration of Sant...
123,186,summarize: allisonpearson you missed the point...,This set of tweets discusses the research into...
124,839,summarize: action alliance heading to court to...,The Action Alliance is heading to court to con...


# ROUGE SCORE

In [7]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [8]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="content", 
                               column_summary="c_summary"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text].tolist(), batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary].tolist(), batch_size))


    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
  
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      

        
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [11]:
rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(t_df, rouge_metric, model_pegasus, tokenizer, column_text = 'content', column_summary='c_summary', batch_size=8)
     

100%|██████████| 16/16 [02:05<00:00,  7.87s/it]


In [12]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.017504,0.000114,0.017068,0.01708


# BERT SCORE

In [13]:
def calculate_BERT_Score_on_test_ds(dataset, model, tokenizer, 
                               batch_size=1, device=device, 
                               column_text="content", 
                               column_summary="c_summary"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text].tolist(), batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary].tolist(), batch_size))
    l = []

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
  
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]
        l.extend(decoded_summaries)
        
    return l

In [14]:
cand_text=calculate_BERT_Score_on_test_ds(t_df, model_pegasus,tokenizer,batch_size=1, column_text = 'content', column_summary='c_summary')
     

100%|██████████| 126/126 [03:36<00:00,  1.72s/it]


In [15]:
len(cand_text)

126

In [16]:
cand_text = [d.replace(".", " ") for d in cand_text]
cand_text = [d.replace("<n>BR>", " ") for d in cand_text]
cand_text = [d.replace("<n>", " ") for d in cand_text]
cand_text = [d.replace("BR>", " ") for d in cand_text]

In [18]:
ref_text = t_df["c_summary"].tolist()
len(ref_text)

126

In [19]:
ref_text = [d.replace(".", " ") for d in ref_text]
ref_text = [d.replace("<n>BR>", " ") for d in ref_text]
ref_text = [d.replace("<n>", " ") for d in ref_text]
ref_text = [d.replace("BR>", " ") for d in ref_text]

In [ ]:
!pip install bert_score

In [22]:
from bert_score import score

# Compute BERT score
P, R, F1 = score(cand_text, ref_text,  lang='en', verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.91 seconds, 43.37 sentences/sec


In [23]:
# Print the calculated scores
print(f'Precision: {P.mean().item():.4f}')
print(f'Recall: {R.mean().item():.4f}')
print(f'F1 Score: {F1.mean().item():.4f}')

Precision: 0.8389
Recall: 0.8437
F1 Score: 0.8409


# FINE TUNING

In [38]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/gggggg/train.csv')
val_df = pd.read_csv('/kaggle/input/gggggg/val.csv')

In [39]:
train_df = train_df.rename(columns={"target_text":"c_summary", "source_text":"content"})
val_df = val_df.rename(columns={"target_text":"c_summary", "source_text":"content"})

In [41]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['content'] , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['c_summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [42]:
from datasets import Dataset

# convert the DataFrame to a Dataset object
train = Dataset.from_pandas(train_df)
val = Dataset.from_pandas(val_df)

train_pt = train.map(convert_examples_to_features, batched = True)
val_pt = val.map(convert_examples_to_features, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3587: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

In [43]:
train_pt[0]

{'Unnamed: 0': 814,
 'content': "summarize: even if the jackals come in front of ussnakes though inside of us.if we take a step backfrom this case.let the sky come in red.the lion of our countryWe will not feed the dog to the coyote..thank you...white partymessiacunacunmessitoilet  happens when you simulate lionel messi's career on football manager 2006, how successful did fm think he was going to be 17 years ago? find out on youtube belowbarca barcelona messi psg laliga sevilla valencia villareal mbappe copadelrey  e stato nominato miglior giocatore dell'anno al fifathebestbenzema, pallone d'oro 2022, he non ha digerito il premio assegnato alla 'pulga' e ha pubblicato una storia. messi ha risposto postando la coppa del mondoe voi, da che parte state? tuttosport  our shouldersmessironaldoearthquakehelpgalatasarayaltaybayindirturkeytekyurekshoulder toturkeyFatih Terimacun  was not surprised about messi winning fifabestawards fifabestplayer of the year 2023shameless fifafifa thebest fifa

In [44]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [45]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-finetune', num_train_epochs=10, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [46]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train_pt, 
                  eval_dataset=val_pt)

In [47]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.419500,0.273555


TrainOutput(global_step=520, training_loss=0.4735219405247615, metrics={'train_runtime': 1667.2358, 'train_samples_per_second': 5.008, 'train_steps_per_second': 0.312, 'total_flos': 8288798320435200.0, 'train_loss': 0.4735219405247615, 'epoch': 9.96})

In [48]:
trainer.save_model('/kaggle/working/')

# TESTING AFTER FINETUNING

# Rouge Score

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_ckpt = "/kaggle/input/pegasus1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [9]:
rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(t_df, rouge_metric, model_pegasus, tokenizer, column_text = 'content', column_summary='c_summary', batch_size=8)
     

100%|██████████| 16/16 [02:04<00:00,  7.77s/it]


In [10]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.012962,0.000206,0.01294,0.012931


# Bert Score 

In [28]:
cand_text=calculate_BERT_Score_on_test_ds(t_df, model_pegasus,tokenizer,batch_size=1, column_text = 'content', column_summary='c_summary')

100%|██████████| 126/126 [03:33<00:00,  1.69s/it]


In [29]:
cand_text = [d.replace(".", " ") for d in cand_text]
cand_text = [d.replace("<n>BR>", " ") for d in cand_text]
cand_text = [d.replace("<n>", " ") for d in cand_text]
cand_text = [d.replace("BR>", " ") for d in cand_text]

In [31]:
ref_text = t_df["c_summary"].tolist()
len(ref_text)

126

In [32]:
ref_text = [d.replace(".", " ") for d in ref_text]
ref_text = [d.replace("<n>BR>", " ") for d in ref_text]
ref_text = [d.replace("<n>", " ") for d in ref_text]
ref_text = [d.replace("BR>", " ") for d in ref_text]

In [33]:
from bert_score import score

# Compute BERT score
P, R, F1 = score(cand_text, ref_text,  lang='en', verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.53 seconds, 49.78 sentences/sec


In [34]:
# Print the calculated scores
print(f'Precision: {P.mean().item():.4f}')
print(f'Recall: {R.mean().item():.4f}')
print(f'F1 Score: {F1.mean().item():.4f}')

Precision: 0.9415
Recall: 0.9350
F1 Score: 0.9381


# COMPARISON BETWEEN ROGUE SCORE AND BERT SCORE

In [ ]:
!pip install bert-score

In [2]:
from bert_score import score

In [90]:
# Example reference and candidate texts
ref_texts = ["The book is written by me"]
cand_text = ["I am the author of that book"]
# Compute BERT score
P, R, F1 = score(cand_text, ref_texts, lang='en', verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.06 seconds, 16.89 sentences/sec


In [91]:
P

tensor([0.9171])

In [92]:
R

tensor([0.9120])

In [93]:
F1

tensor([0.9146])

In [94]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score("The book is written by me",
                      "I am the author of that book")

In [95]:
scores

{'rouge1': Score(precision=0.2857142857142857, recall=0.3333333333333333, fmeasure=0.30769230769230765),
 'rougeL': Score(precision=0.2857142857142857, recall=0.3333333333333333, fmeasure=0.30769230769230765)}

In [97]:
rouge_metric = load_metric('rouge')

In [101]:
rouge_metric = load_metric('rouge')
 
rouge_metric.add_batch(predictions=["The book is written by me"], references=["I am the author of that book"])
        
    #  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()

In [103]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['scores'])

,rouge1,rouge2,rougeL,rougeLsum
scores,0.307692,0.0,0.307692,0.307692


# FINETUNING T5

In [35]:
import pandas as pd
df = pd.read_csv('/kaggle/input/NLPSUM/Training.csv')
df = df[0:983]

In [36]:
df

,Unnamed: 0,Unnamed: 0.1,content,c_summary
0,0,0,exciting news from openai they have just relea...,chatgptOpenAI has released their ChatGPT and W...
1,1,1,the first words i typed this morning git fetc...,This set of tweets discusses the recent launch...
2,2,2,i morning briefing how are universities dealin...,This set of tweets is discussing the use of Op...
3,3,3,the speed at with microsoft is executing on o...,"This set of tweets is discussing OpenAI, a com..."
4,4,4,openai integration tools for odoo editor are n...,This set of tweets discusses the integration o...
...,...,...,...,...
978,978,978,bentocaipira hahahahahawelcome to the club do ...,This set of tweets discusses the changing perc...
979,979,979,demain live minimum 4h 2 clip twitter car je ...,Tomorrow there will be a live stream for at le...
980,980,980,a pior coisa faith eu lose meu cartao vt justo...,This set of tweets discusses a variety of topi...
981,981,981,ai se on muuttunu jo noin ovelaksi toi nuha......,This group of tweets is discussing a variety o...


In [3]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"c_summary":"target_text", "content":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: exciting news from openai they have...,chatgptOpenAI has released their ChatGPT and W...
1,summarize: the first words i typed this mornin...,This set of tweets discusses the recent launch...
2,summarize: i morning briefing how are universi...,This set of tweets is discussing the use of Op...
3,summarize: the speed at with microsoft is exe...,"This set of tweets is discussing OpenAI, a com..."
4,summarize: openai integration tools for odoo e...,This set of tweets discusses the integration o...
...,...,...
978,summarize: bentocaipira hahahahahawelcome to t...,This set of tweets discusses the changing perc...
979,summarize: demain live minimum 4h 2 clip twit...,Tomorrow there will be a live stream for at le...
980,summarize: a pior coisa faith eu lose meu cart...,This set of tweets discusses a variety of topi...
981,summarize: ai se on muuttunu jo noin ovelaksi ...,This group of tweets is discussing a variety o...


In [4]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.16)
train_df.shape, test_df.shape

((825, 2), (158, 2))

In [5]:
val_df, test_df = train_test_split(test_df, test_size=0.5)

In [6]:
val_df.shape, test_df.shape

((79, 2), (79, 2))

In [ ]:
!pip install SimpleT5

In [8]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

# Testing Before Finetuning

In [10]:
import torch

model.device= torch.device("cpu")

In [11]:
def truncate_input(text, max_length):
    """
    Truncates input text to a specified maximum length.

    Args:
        text (str): The input text to be truncated.
        max_length (int): The maximum length of the truncated text.

    Returns:
        str: The truncated text.
    """
    if len(text) <= max_length:
        return text
    else:
        return text[:max_length]

In [12]:
pred = []
for i in test_df['source_text']:
    pred.extend(model.predict(truncate_input(i, 512)))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
test_df

,source_text,target_text
645,summarize: To women Ladli Bahna YojanaTo daugh...,The Madhya Pradesh government has made a numbe...
440,summarize: Janara Melee Bane Baney Ausher Hawr...,This set of tweets is discussing the recent hi...
771,summarize: madefortrademadefortrademadefortrad...,This set of tweets is promoting the Made For T...
552,summarize: bodhdiwasofsantgaribdasji3 days lef...,Bodh Diwas of Saint Garibdas Ji Maharaj Chudan...
758,summarize: madefortrade i 7 din mein paise do...,This set of tweets is discussing a variety of ...
...,...,...
162,summarize: joeridecae marcbonten yes I think a...,This set of tweets is discussing various topic...
224,summarize: justin update chandrayaan3The fligh...,ISRO has achieved a major success in their pla...
341,summarize: indvaus the way in which the aussie...,This set of tweets is about the India vs Austr...
21,summarize: openai investment thesis map start...,This set of tweets is about OpenAI and its inv...


In [17]:
test_df['source_text'][758]

"summarize: madefortrade  i 7 din mein paise double possible? impossible? watchindvsausbgt2023upgoesglobalchocolatedaybordergavaskartrophy  control of your organization's future by investing in sap risk management today with our comprehensive and userfriendly solution, you can identify, evaluate, and mitigate risk in realtime. visit .. madefortrade riskmanagement grcsidrabankk madefortrade bbb23 coredaoorg   core adress is 0x2a399952b9abb76cb9f1ec2c4d853e35f0e4bddfno more trades, will see if we get a good opportunity towards the last hour or so. as a trader it is important to know when to step aside even if that means a lost opportunity. madefortradeindiancert the officials of the tamil nadu government are using gmail services against the email policy 2014 of the govt. of india. are they extra  constitutional authorities? madefortrade mrunalthakurpriyankachaharchoudhary  is the best online mba colleges in punjab procedurefor more details madefortrade bgt2023 jadeja eknathshinde ourpm b

In [18]:
test_df['target_text'][758]

'This set of tweets is discussing a variety of topics, including investing in SAP risk management, trading and knowing when to step aside, the Tamil Nadu government using Gmail services against the email policy of India, online MBA colleges in Punjab, and inflation in India. It also includes references to Instagram being down, God, and the importance of remembering our real home.'

In [19]:
pred[4]

'madefortrade bbb23 coredaoorg core adress is 0x2a399952b9abb76cb9f1ec2c4d853e35f0e4bddfno more trades, will see if we get a good op.'

In [20]:
len(pred)

79

In [21]:
ref_text = test_df['target_text'].tolist()

len(ref_text)

79

# BERT SCORE

In [24]:
from bert_score import score

# Compute BERT score
P, R, F1 = score(pred, ref_text,  lang='en', verbose=True)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 3.18 seconds, 24.83 sentences/sec


In [25]:
# Print the calculated scores
print(f'Precision: {P.mean().item():.4f}')
print(f'Recall: {R.mean().item():.4f}')
print(f'F1 Score: {F1.mean().item():.4f}')

Precision: 0.8376
Recall: 0.8339
F1 Score: 0.8351


# ROUGE SCORE

In [ ]:
!pip install rouge_score

In [27]:
from datasets import load_dataset, load_metric
rouge_metric = load_metric('rouge')

 
rouge_metric.add_batch(predictions=pred, references=ref_text)
        
    #  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()

In [28]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['scores'])

,rouge1,rouge2,rougeL,rougeLsum
scores,0.210107,0.082602,0.15543,0.155516


# Finetuning

In [29]:
model.train(train_df=train_df,
            eval_df=val_df, 
            source_max_token_len=528, 
            target_max_token_len=300, 
            batch_size=4, 
            max_epochs=10, 
            use_gpu=True
           )

Validation sanity check: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Training: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Validating: 0it [00:00, ?it/s]

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validating: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Validating: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
cd /kaggle/working/outputs


/kaggle/working/outputs


In [31]:
ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
simplet5-epoch-0-train-loss-1.8575-val-loss-1.3511/
simplet5-epoch-1-train-loss-1.3781-val-loss-1.3076/
simplet5-epoch-2-train-loss-1.1773-val-loss-1.2924/
simplet5-epoch-3-train-loss-1.0321-val-loss-1.322/
simplet5-epoch-4-train-loss-0.8982-val-loss-1.3354/
simplet5-epoch-5-train-loss-0.7902-val-loss-1.3608/
simplet5-epoch-6-train-loss-0.7062-val-loss-1.4282/
simplet5-epoch-7-train-loss-0.6183-val-loss-1.4629/
simplet5-epoch-8-train-loss-0.5558-val-loss-1.5132/
simplet5-epoch-9-train-loss-0.4929-val-loss-1.5747/


In [ ]:
!tar -zcvf oy.tar.gz /kaggle/working

# Testing After Finetuning

In [32]:
# let's load the trained model for inferencing:
model.load_model("t5","simplet5-epoch-2-train-loss-1.1773-val-loss-1.2924", use_gpu=True)

In [33]:
pred = []
for i in test_df['source_text']:
    pred.extend(model.predict(truncate_input(i, 512)))

In [34]:
test_df

,source_text,target_text
645,summarize: To women Ladli Bahna YojanaTo daugh...,The Madhya Pradesh government has made a numbe...
440,summarize: Janara Melee Bane Baney Ausher Hawr...,This set of tweets is discussing the recent hi...
771,summarize: madefortrademadefortrademadefortrad...,This set of tweets is promoting the Made For T...
552,summarize: bodhdiwasofsantgaribdasji3 days lef...,Bodh Diwas of Saint Garibdas Ji Maharaj Chudan...
758,summarize: madefortrade i 7 din mein paise do...,This set of tweets is discussing a variety of ...
...,...,...
162,summarize: joeridecae marcbonten yes I think a...,This set of tweets is discussing various topic...
224,summarize: justin update chandrayaan3The fligh...,ISRO has achieved a major success in their pla...
341,summarize: indvaus the way in which the aussie...,This set of tweets is about the India vs Austr...
21,summarize: openai investment thesis map start...,This set of tweets is about OpenAI and its inv...


In [39]:
test_df['source_text'][224]

"summarize: justin update chandrayaan3The flight acceptance hottest of ce20 engine that will power the cus of lvm3 gslvmk3 launch vehicle for chandrayaan3 mission was successfully conducted on February 24, 2023 at propulsion complex, Mahendragiri isro httpst. conurghfqe7ngreat news   isro has successfully performed a 25 second flight acceptance test of the ce20 cryogenicengine that will power the third stage of the lvm3 vehicle that will launch chandrayaan3   nagarsahyadri india chandrayaan3 isrochandrayaan3 ISRO gets big success, Chandrayaan3's cryogenic engine test successfulisro chandrayaan3 has successfully performed a 25 second flight acceptance test of the ce20 cryogenic engine that will power the third stage of the lvm3 vehicle that will launch chandrayaan3.  has successfully completed the flight acceptance test of the ce20 engine that will be used on the lvm3m4chandrayaan3 mission it was a 25 sec duration test carried out on 24 feb.a julaug '23 launch of cy3 is still very much 

In [40]:
pred[75]

'The Indian Space Research Organisation ISRO has successfully performed a 25 second flight acceptance test of the CE20 cryogenicengine that will power the third stage of the LVM3 launch vehicle for the Chandrayaan3 mission on February 24, 2023 at Mahendragiri Propulsion Complex, Mahendragiri. This test was successful and is expected to be completed by the end of 2023.'

In [41]:
ref_text = test_df['target_text'].tolist()

len(ref_text)

79

In [42]:
ref_text[75]

'ISRO has achieved a major success in their plans for the Chandrayaan3 mission, with a successful 25 second flight acceptance test of the CE20 cryogenic engine that will power the third stage of the launch vehicle. The test was conducted on February 24, 2023 and the mission is still on track for a launch in JulyAugust of 2023.'

# BERT SCORE

In [43]:
from bert_score import score
P, R, F1 = score(pred, ref_text,  lang='en', verbose=True)
# Print the calculated scores
print(f'Precision: {P.mean().item():.4f}')
print(f'Recall: {R.mean().item():.4f}')
print(f'F1 Score: {F1.mean().item():.4f}')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.74 seconds, 45.47 sentences/sec
Precision: 0.8887
Recall: 0.8787
F1 Score: 0.8833


# ROGUE SCORE

In [44]:
from datasets import load_dataset, load_metric
rouge_metric = load_metric('rouge')

 
rouge_metric.add_batch(predictions=pred, references=ref_text)
        
    #  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()

In [45]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['scores'])

,rouge1,rouge2,rougeL,rougeLsum
scores,0.419542,0.20525,0.317355,0.317625


# FINETUNING BART

In [7]:
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

In [8]:
# ----- Torch Device ----- #
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
# --- DEFINE MODEL AND TOKENIZER --- #
model_name = "facebook/bart-large"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [10]:
from torch import nn 
# ---- Freeze parameters

def freeze_params(model: nn.Module):
    """Set requires_grad=False for each of model.parameters()"""
    for par in model.parameters():
        par.requires_grad = False

In [11]:
def freeze_embeds(model):
    """Freeze token embeddings and positional embeddings for BART and PEGASUS, just token embeddings for t5."""
    model_type = model.config.model_type
    if model_type == "t5":
        freeze_params(model.shared)
        for d in [model.encoder, model.decoder]:
            freeze_params(d.embed_tokens)
    else:
        freeze_params(model.model.shared)
        for d in [model.model.encoder, model.model.decoder]:
            freeze_params(d.embed_positions)
            freeze_params(d.embed_tokens)


In [12]:
df

,source_text,target_text
0,summarize: exciting news from openai they have...,chatgptOpenAI has released their ChatGPT and W...
1,summarize: the first words i typed this mornin...,This set of tweets discusses the recent launch...
2,summarize: i morning briefing how are universi...,This set of tweets is discussing the use of Op...
3,summarize: the speed at with microsoft is exe...,"This set of tweets is discussing OpenAI, a com..."
4,summarize: openai integration tools for odoo e...,This set of tweets discusses the integration o...
...,...,...
978,summarize: bentocaipira hahahahahawelcome to t...,This set of tweets discusses the changing perc...
979,summarize: demain live minimum 4h 2 clip twit...,Tomorrow there will be a live stream for at le...
980,summarize: a pior coisa faith eu lose meu cart...,This set of tweets discusses a variety of topi...
981,summarize: ai se on muuttunu jo noin ovelaksi ...,This group of tweets is discussing a variety o...


In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15)

In [14]:
test

,source_text,target_text
740,summarize: madefortrade17 february 2023 17 feb...,"This set of tweets is about trading, entrepren..."
372,summarize: careerbest firstclass figures for m...,This set of tweets is about the India vs Austr...
2,summarize: i morning briefing how are universi...,This set of tweets is discussing the use of Op...
323,summarize: mixed day for r ashwinbecame no.1 b...,This set of tweets is discussing the performan...
657,summarize: Mukhyamantri Ladli Bahna Yojana dep...,This budget in Madhya Pradesh is dedicated to ...
...,...,...
913,summarize: When I looked up I thought it was h...,This set of tweets is about a variety of topic...
7,summarize: news just in from openai chatgpt a...,ziOpenAI has announced the release of two new ...
330,summarize: ind vs aus 2023 indvaus cr...,bgt2023This set of tweets is about the India v...
383,summarize: indvaus indvsaustest bordergavaskar...,This set of tweets discusses the India vs. Aus...


In [15]:
val, test = train_test_split(test, test_size=0.85)

In [16]:
val

,source_text,target_text
40,summarize: mai has been incredible thus far. w...,"aThis set of tweets is about OpenAI's chatGPT,..."
385,summarize: nature at its bestindvaus celinebyh...,This set of tweets is about cricket and relate...
880,summarize: rema holiday listen live via e...,This set of tweets is about a variety of topic...
515,"summarize: bodhdiwasofsantgaribdasjiPoor, came...",", an auspicious occasion to celebrate the life..."
524,summarize: bodhdiwasofsantgaribdasjiGod Kabir ...,of Saint Garibdas Ji Maharaj. Kabir Saheb has ...
118,summarize: rabaukenmamad sarinchenoo docnrw bi...,These tweets discuss the origins of SARSCoV2 a...
277,summarize: chandrayaan3isro is gearing up to l...,The Indian Space Research Organization ISRO is...
805,summarize: metaversevrtoken team is amazing bi...,This set of tweets discusses the potential for...
126,summarize: warclandestine boston university al...,This set of tweets discusses the origin of SAR...
592,summarize: bodhdiwasofsantgaribdasji3 days lef...,This set of tweets is about the upcoming Bodh ...


In [17]:
val.to_csv('val.csv')
train.to_csv('train.csv')
test.to_csv('test.csv')

In [18]:
test

,source_text,target_text
164,summarize: collinrugg sars cov 2 was planned....,This set of tweets discusses the origin of SAR...
492,summarize: Where are you Smriti Irani...??Nare...,This set of tweets is discussing the recent in...
323,summarize: mixed day for r ashwinbecame no.1 b...,This set of tweets is discussing the performan...
898,summarize: it is obvious messi winning the nex...,This set of tweets is discussing Lionel Messi'...
732,summarize: the armed forces understand that th...,"This set of tweets discusses the stock market,..."
...,...,...
209,summarize: ISRO has successfully completed fli...,The Indian Space Research Organisation ISRO ha...
576,summarize: bodhdiwasofsantgaribdasji Kabir Sag...,These tweets are about the celebration of Sant...
186,summarize: allisonpearson you missed the point...,This set of tweets discusses the research into...
839,summarize: action alliance heading to court to...,The Action Alliance is heading to court to con...


In [19]:
train

,source_text,target_text
814,summarize: even if the jackals come in front o...,kThis set of tweets is about Lionel Messi and ...
790,"summarize: Throughout this blog, we will under...",This set of tweets is about the challenges of ...
167,summarize: tomfitton we've all known it origin...,This set of tweets discusses the origin of the...
328,"summarize: morning session becomes crucial, no...",This set of tweets is about the 3rd Test betwe...
667,summarize: Keeping in mind the welfare of ever...,This set of tweets discusses the budget of Mad...
...,...,...
35,summarize: robots and ai will take some jobs a...,This set of tweets discusses the potential imp...
894,summarize: Bosso Confident Flying Eagles Will ...,"This set of tweets covers a variety of topics,..."
79,"summarize: When I saw the original suggestion,...",This set of tweets discusses the use of OpenAI...
228,summarize: The Chandrayaan3 spacecraft has cle...,ISRO has been working on the Chandrayaan3 spac...


In [22]:
# ----- Reading in the Dataset
raw_datasets = load_dataset('csv', data_files={'train': '/kaggle/working/train.csv',
                                          'valid': '/kaggle/working/val.csv',
                                          'test': '/kaggle/working/test.csv'})


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6f86c46d99e77180/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(examples["source_text"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3587: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    output_dir = '/kaggle/working/results/bart', 
    logging_dir = '/kaggle/working/logs/bart'
)
     

In [ ]:
!pip install datasets
!pip install sentencepiece
!pip install transformers
!pip install rouge_score
!pip install bert_score 

from bert_score import score
from datasets import load_dataset, Dataset, load_metric
import sys
#from google.colab import drive
import pandas as pd
import numpy as np
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import huggingface_hub
import matplotlib.pyplot as plt
import nltk
nltk.download("punkt")
#import gc
from torch import nn 

In [25]:
metric = load_metric("rouge")

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}
     

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.093449,25.971700,17.074000,23.354800,24.778700,20.000000
2,No log,1.033661,24.374400,16.182700,22.348000,23.376700,20.000000
3,1.207600,1.036758,24.674500,16.700100,22.528700,23.845400,20.000000


TrainOutput(global_step=627, training_loss=1.1736385461055872, metrics={'train_runtime': 426.6191, 'train_samples_per_second': 5.872, 'train_steps_per_second': 1.47, 'total_flos': 2523366863659008.0, 'train_loss': 1.1736385461055872, 'epoch': 3.0})

In [31]:
!tar -zcvf oy.tar.gz /kaggle/working

tar: Removing leading `/' from member names
/kaggle/working/
/kaggle/working/.virtual_documents/
/kaggle/working/logs/
/kaggle/working/logs/bart/
/kaggle/working/logs/bart/1681474584.7286136/
/kaggle/working/logs/bart/1681474584.7286136/events.out.tfevents.1681474584.23618fd07ff5.23.3
/kaggle/working/logs/bart/events.out.tfevents.1681474584.23618fd07ff5.23.2
/kaggle/working/logs/bart/events.out.tfevents.1681474250.23618fd07ff5.23.0
/kaggle/working/logs/bart/1681474250.4988203/
/kaggle/working/logs/bart/1681474250.4988203/events.out.tfevents.1681474250.23618fd07ff5.23.1
/kaggle/working/val.csv
/kaggle/working/test.csv
/kaggle/working/wandb/
/kaggle/working/wandb/debug-internal.log
/kaggle/working/wandb/debug.log
/kaggle/working/wandb/run-20230414_121115-fk1sdj2t/
/kaggle/working/wandb/run-20230414_121115-fk1sdj2t/logs/
/kaggle/working/wandb/run-20230414_121115-fk1sdj2t/logs/debug-internal.log
/kaggle/working/wandb/run-20230414_121115-fk1sdj2t/logs/debug.log
/kaggle/working/wandb/run-202

In [ ]:
# --------------------- # 
#    TEST EVALUATION    #
# --------------------- #

out = trainer.predict(tokenized_datasets["test"])
generated_summaries = []
for i in range(0, 110): 
  generated_summaries.append(tokenizer.decode(out[0][i], skip_special_tokens =  True))
ground_truth = tokenized_datasets["test"]["summaries"]
conversation = tokenized_datasets["test"]["inputs"]

In [33]:
# --------------------- # 
#    TEST EVALUATION    #
# --------------------- #

out = trainer.predict(tokenized_datasets["test"])
generated_summaries = []
for i in range(0, 126): 
  generated_summaries.append(tokenizer.decode(out[0][i], skip_special_tokens =  True))
ground_truth = tokenized_datasets["test"]["target_text"]
conversation = tokenized_datasets["test"]["source_text"]

In [34]:
out.metrics

{'test_loss': 1.1961466073989868,
 'test_rouge1': 26.0946,
 'test_rouge2': 16.0003,
 'test_rougeL': 23.2339,
 'test_rougeLsum': 24.7662,
 'test_gen_len': 20.0,
 'test_runtime': 22.5724,
 'test_samples_per_second': 5.582,
 'test_steps_per_second': 1.418}

In [36]:
from bert_score import score

# Compute BERT score
P, R, F1 = score(generated_summaries, ground_truth,  lang='en', verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.87 seconds, 67.25 sentences/sec


In [37]:
# Print the calculated scores
print(f'Precision: {P.mean().item():.4f}')
print(f'Recall: {R.mean().item():.4f}')
print(f'F1 Score: {F1.mean().item():.4f}')

Precision: 0.9151
Recall: 0.8415
F1 Score: 0.8765
